In [1]:
# Общие библиотеки
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
import json
from math import sin, cos

# Для создания и обучения модели
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms

# Для работы с изображениями
import cv2
from PIL import Image

# Для визуализации
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_to_model_weights1 = "drive/MyDrive/Colab Notebooks/db2022/content/model/model_1_alld.pth"
path_to_model_weights2 = "drive/MyDrive/Colab Notebooks/db2022/content/model/model_2_alld.pth"

In [4]:
class ImageDataset(Dataset):
    def __init__(self, data_df, transform=None):

        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name, labels = self.data_df.iloc[idx]['id'], [self.data_df.iloc[idx]['left_top_x']/10496, 
                                                            self.data_df.iloc[idx]['left_top_y']/10496, 
                                                            self.data_df.iloc[idx]['right_bottom_x']/10496, 
                                                            self.data_df.iloc[idx]['right_bottom_y']/10496, 
                                                            self.data_df.iloc[idx]['angle']/360]

        #print(image_name)  
        # читаем картинку. read the image
        image = cv2.imread(f"drive/MyDrive/Colab Notebooks/db2022/content/train/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно. transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(labels).float()
    
    def __len__(self):
        return len(self.data_df)

In [5]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

In [6]:
from os import listdir

print("Обучающей выборки " ,len(listdir("drive/MyDrive/Colab Notebooks/db2022/content/train")))
print("Тестовой выборки " ,len(listdir("drive/MyDrive/Colab Notebooks/db2022/content/test")))

#Обучающей выборки 800
#Тестовой выборки 400

Обучающей выборки  800
Тестовой выборки  400


In [7]:
# разделим датасет на трейн и валидацию, чтобы смотреть на качество
#train_df, valid_df = train_test_split(data_df, test_size=0.2, random_state=43)

In [8]:
#train_dataset = ImageDataset(train_df, train_transform)
#valid_dataset = ImageDataset(valid_df, valid_transform)

In [9]:
batch_size= 16

In [10]:
#train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                           batch_size= batch_size,
#                                           shuffle=True,
#                                           pin_memory=True,
#                                           num_workers=2)

#valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
#                                           batch_size= batch_size,
#                                           # shuffle=True,
#                                           pin_memory=True,
#                                           num_workers=2)

In [11]:
def compute_metric(data_true, data_pred, outImageW = 10496, outImageH = 10496):

    x_center_true = np.array((data_true[0] + data_true[2])/2).astype(int)
    y_center_true = np.array((data_true[1] + data_true[3])/2).astype(int)

    x_metr = x_center_true - np.array((data_pred[0] + data_pred[2])/2).astype(int)
    y_metr = y_center_true - np.array((data_pred[1] + data_pred[3])/2).astype(int)

    metr =  1 - 0.7 * (abs(x_metr)/outImageH + abs(y_metr)/outImageW)/2 + 0.3 *abs(data_pred[4] - data_true[4])/359
    return metr

In [12]:
def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)
    
    points = np.array(val_history)
    steps = list(range(0, len(train_history) + 1, int(len(train_history) / len(val_history))))[1:]
    
    plt.scatter(steps, val_history, marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    
    plt.legend(loc='best')
    plt.grid()

    plt.show()

In [13]:
def train(res_model, criterion, optimizer, train_dataloader, test_dataloader, NUM_EPOCH, lr_scheduler):
    train_loss_log = []
    val_loss_log = []
    
    train_acc_log = []
    val_acc_log = []
    
    for epoch in tqdm(range(NUM_EPOCH)):
        model.train()
        train_loss = 0.
        train_size = 0
        
        train_pred = []

        for imgs, labels in train_dataloader:
            optimizer.zero_grad()

            imgs = imgs.cuda()
            labels = labels.cuda()

            y_pred = model(imgs)

            loss = criterion(y_pred, labels)
            loss.backward()
            
            train_loss += loss.item()
            train_size += y_pred.size(0)
            train_loss_log.append((loss.data.cpu().detach().numpy() / y_pred.size(0)) * 100)
            
            y_pred[:, :4] = y_pred[:, :4] * 10496
            y_pred[:, -1] = y_pred[:, -1] * 360

            labels[:, :4] = labels[:, :4] * 10496
            labels[:, -1] = labels[:, -1] * 360

            for label, pr in zip(labels, y_pred):
                    train_pred.append(compute_metric(label.cpu().detach().numpy(), pr.cpu().detach().numpy()))

            optimizer.step()

        train_acc_log.append(train_pred)

        epoch_train_loss = (train_loss / train_size)*100
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_train_loss
            }, path_to_model_weights)



        val_loss = 0.
        val_size = 0
        
        val_pred = []
        
        lr_scheduler.step()


        model.eval()
        
        with torch.no_grad():
            for imgs, labels in test_dataloader:
                
                imgs = imgs.cuda()
                labels = labels.cuda()
                
                pred = model(imgs)
                loss = criterion(pred, labels)

                pred[:, :4] = pred[:, :4] * 10496
                pred[:, -1] = pred[:, -1] * 360

                labels[:, :4] = labels[:, :4] * 10496
                labels[:, -1] = labels[:, -1] * 360
                
                val_loss += loss.item()
                val_size += pred.size(0)

                for label, pr in zip(labels, pred):
                    val_pred.append(compute_metric(label.cpu().detach().numpy(), pr.cpu().detach().numpy()))

        val_loss_log.append((val_loss / val_size)*100)
        val_acc_log.append(val_pred)

        clear_output()
        plot_history(train_loss_log, val_loss_log, 'loss')
        


        print('Train loss:', (train_loss / train_size)*100)
        print('Val loss:', (val_loss / val_size)*100)
        print('Train metric:', (np.mean(train_pred)))
        print('Val metric:', (np.mean(val_pred)))
        
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

In [14]:
torch.cuda.empty_cache()

# Подргружаем модель

#model = models.resnet50(pretrained=True)
#model = models.resnet152(pretrained=True)
#model = models.wide_resnet50_2(pretrained=True)
#model = models.wide_resnet101_2(pretrained=True)
#model = models.resnet101(pretrained=True)


#model = models.resnet101(pretrained=True)

#model = models.googlenet(pretrained=True)    не запускается  RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x1024 and 2048x5)
#model =  models.inception_v3(pretrained=True) не запускается Calculated padded input size per channel: (3 x 3). Kernel size: (5 x 5). Kernel size can't be greater than actual input size
#model = models.densenet161(pretrained=True)  не запускается RuntimeError: The size of tensor a (1000) must match the size of tensor b (5) at non-singleton dimension 1
#model = models.squeezenet1_0(pretrained=True) не запускается RuntimeError: The size of tensor a (1000) must match the size of tensor b (5) at non-singleton dimension 1
#model = models.shufflenet_v2_x1_0(pretrained=True) не запускается  RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x1024 and 2048x5)
#model = models.mobilenet_v2(pretrained=True) не запускается RuntimeError: The size of tensor a (1000) must match the size of tensor b (5) at non-singleton dimension 1
#model = models.mnasnet1_0(pretrained=True) не запускается RuntimeError: The size of tensor a (1000) must match the size of tensor b (5) at non-singleton dimension 1
#model = models.vgg16(pretrained=True) не запускаются RuntimeError: The size of tensor a (1000) must match the size of tensor b (5) at non-singleton dimension 1
#model = models.resnet18(pretrained=True) не запускается 



model_1 = models.resnet101(pretrained=True)
model_1.fc = nn.Linear(2048, 5)
#model_1 = model_1.cuda()





model_2 = models.resnext50_32x4d(pretrained=True)
model_2.fc = nn.Linear(2048, 5)
#model_2 = model_2.cuda()


#model.fc = nn.Linear(2048, 5)
#model = model.cuda()

#criterion =  nn.SmoothL1Loss()

#optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9, verbose=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

In [15]:
checkpoint_1 = torch.load(path_to_model_weights1, map_location=torch.device('cpu'))
model_1.load_state_dict(checkpoint_1['model_state_dict'])
#model_1.to(device)

checkpoint_2 = torch.load(path_to_model_weights2, map_location=torch.device('cpu'))
model_2.load_state_dict(checkpoint_2['model_state_dict'])
#model_2.to(device)

<All keys matched successfully>

In [16]:
class TestImageDataset(Dataset):
    def __init__(self, files, transform=None):

        self.files = files
        self.transform = transform

    def __getitem__(self, idx):

        image_name = self.files[idx]

        # читаем картинку. read the image
        image = cv2.imread(f"drive/MyDrive/Colab Notebooks/db2022/content/test/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно. transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image
    
    def __len__(self):
        return len(self.files)

In [17]:
test_images_dir = 'drive/MyDrive/Colab Notebooks/db2022/content/test/'

for _, _, test_files in os.walk(test_images_dir):
    
    break

test_dataset = TestImageDataset(test_files, valid_transform)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size= batch_size,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=2
                                           )

In [18]:
indexes = [x.split('.')[0] for x in test_files]
preds_1 = []
preds_2 = []

for imgs in test_loader:
    #imgs = imgs.cuda()            
    pred_1 = model_1(imgs)
    pred_1 = pred_1.cpu().detach().numpy()
    pred_1[:, :4] = np.clip(pred_1[:, :4] * 10496, 0, 10496)
    pred_1[:, -1] = np.clip(pred_1[:, -1] * 360, 0, 360)
    preds_1.extend(list(pred_1))


    pred_2 = model_2(imgs)
    pred_2 = pred_2.cpu().detach().numpy()
    pred_2[:, :4] = np.clip(pred_2[:, :4] * 10496, 0, 10496)
    pred_2[:, -1] = np.clip(pred_2[:, -1] * 360, 0, 360)
    preds_2.extend(list(pred_2))

preds = []

for i in range(len(preds_1)):
  preds.append([ (preds_1[i][0]+preds_2[i][0])/2, (preds_1[i][1]+preds_2[i][1])/2, (preds_1[i][2]+preds_2[i][2])/2, (preds_1[i][3]+preds_2[i][3])/2, (preds_1[i][4]+preds_2[i][4])/2  ])

In [19]:
sub_dir = "drive/MyDrive/Colab Notebooks/db2022/content/submission_blend_all_d/"
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)

json_true = []

for indx, pred in zip(indexes, preds):

    pred = [int(x) for x in pred]

    left_top = [pred[0], pred[1]]
    right_top = [pred[2], pred[1]]
    left_bottom = [pred[0], pred[3]]
    right_bottom = [pred[2], pred[3]]
        
    res = {
        'left_top': left_top,
        'right_top': right_top,
        'left_bottom': left_bottom,
        'right_bottom': right_bottom,
        'angle': pred[4]
    }

    with open(sub_dir+indx+'.json', 'w') as f:
        json.dump(res, f)